In [1]:

! pip install datasets



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [13]:
import torch

In [2]:
from transformers import pipeline
bert_ckpt="transformersbook/bert-base-uncased-finetuned-clinc"
pipe=pipeline("text-classification",model=bert_ckpt)

In [3]:
# test to get the confidence of the model
query=""" Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in
Paris and I need a 15 passenger van
"""
print(pipe(query))

[{'label': 'car_rental', 'score': 0.5490034222602844}]


In [4]:
# performance benchmark class
class PerformanceBenchmark:
  def __init__(self,pipeline,dataset,optim_type="BERT baseline"):
    self.pipeline=pipeline
    self.dataset=dataset
    self.optim_type=optim_type
  def compute_accuracy(self):
    pass
  def compute_size(self):
    pass
  def time_pipeline(self):
    pass
  def run_benchmark(self):
    metrics={}
    metrics[self.optim_type]=self.compute_size()
    metrics[self.optim_type].update(self.time_pipeline())
    metrics[self.optim_type].update(self.compute_accuracy())
    return metrics

In [5]:
# computing the model accuracy on the test dataset
from datasets import load_dataset
clinc = load_dataset("clinc_oos","plus")

In [6]:
sample=clinc["test"][30]
sample

{'text': 'can you please provide me with assistance in moving money from one account to another',
 'intent': 133}

In [7]:
# mapping the IDs to intents
intents=clinc["test"].features["intent"]
intents.int2str(sample["intent"])

'transfer'

In [9]:
# will use the accuracy metric as the dataset is distributed across the intents
from datasets import load_metric
accuracy_score=load_metric('accuracy', trust_remote_code=True)


In [20]:
# method to calculate the accuracy
def compute_accuracy(self):
  """This method overrides the PerformanceBenchmark.compute_accuracy() method"""
  preds,labels=[],[]
  for example in self.dataset:
    pred=self.pipeline(example["text"])[0]["label"]
    label= example["intent"]
    preds.append(intents.str2int(pred))
    labels.append(label)
  accuracy=accuracy_score.compute(predictions=preds, references=labels)
  print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
  return accuracy
PerformanceBenchmark.compute_accuracy=compute_accuracy

In [11]:
# compute the size of our model using torch.save()
# in pytorch the prefered way to save a model is state_dict() that maps each layer to its learnable parameters( weights and biases)
list(pipe.model.state_dict().items())[42]

('bert.encoder.layer.2.attention.self.value.bias',
 tensor([-2.7834e-02,  4.9434e-02,  8.3551e-02,  4.1092e-02,  6.0157e-01,
          1.1774e-01, -5.2112e-02, -6.5143e-02, -2.9358e-02, -4.2250e-02,
          7.9177e-02,  8.0409e-02,  2.9921e-03,  1.7816e-01, -5.0480e-02,
         -1.5634e-01, -2.1707e-02,  1.4381e-02,  2.5132e-02, -2.4110e-02,
         -1.9183e-01, -7.8657e-02,  5.0709e-02,  3.3632e-02, -3.1946e-02,
          1.1616e-01,  9.2720e-02, -1.1787e-01,  2.3233e-01, -1.2678e-02,
         -1.3138e-01, -4.0024e-02,  7.4823e-02, -5.4148e-02, -1.5184e-01,
         -7.4407e-02,  1.1559e-01,  8.2729e-02, -1.3787e-01,  8.3528e-02,
          1.2154e-01,  1.6880e-02, -5.6629e-02, -3.9295e-02,  5.3725e-02,
          6.8602e-02, -1.1294e-01,  4.4001e-02, -2.5884e-01,  1.6767e-01,
          1.8316e-01,  5.6272e-02, -3.6874e-02, -2.7938e-02, -9.3204e-02,
         -7.5239e-03,  4.1141e-02, -1.1542e-02, -9.9749e-02, -3.0910e-02,
          4.1398e-02, -4.4389e-02, -2.6279e-02,  7.2100e-02, 

In [14]:
# saving the model
torch.save(pipe.model.state_dict(),"model.pt")

In [19]:
# getting the model size
from pathlib import Path
def compute_size(self):
  '''This overrides the PerformanceBenchmark.compute_size() method'''
  state_dict=self.pipeline.model.state_dict()
  tmp_path=Path("model.pt")
  torch.save(state_dict,tmp_path)
  # calculate the size in megabytes
  size_mb= Path(tmp_path).stat().st_size /(1024 * 1024)
  # delete the temporary file
  tmp_path.unlink()
  print(f" Model size (MB) - {size_mb:.2f}")
  return {"size_mb": size_mb}
PerformanceBenchmark.compute_size=compute_size

In [15]:
# get the execution speed by passing the test query and counting the time between the start and the end
from time import perf_counter
for _ in range(3):
  start_time=perf_counter()
  _=pipe(query)
  latency=perf_counter() - start_time
  print(f"Latency (ms) - {1000 * latency:.3f}")



Latency (ms) - 232.629
Latency (ms) - 129.348
Latency (ms) - 133.744


In [18]:
# perform some operations
import numpy as np
def time_pipeline(self,query="What is the pin number for my account?"):
  ''' This overrides the PerformanceBenchmark.time_pipeline() method'''
  latencies=[]
  # Warmup CPU
  for _ in range(10):
    _=self.pipeline(query)
  # Timed run
  for _ in range(100):
    start_time=perf_counter()
    _=self.pipeline(query)
    latency=perf_counter() - start_time
    latencies.append(latency)
  # Compute run statistics
  time_avg_ms =1000 * np.mean(latencies)
  time_std_ms= 1000 * np.std(latencies)
  print(f"Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}")
  return {"time_avg_ms": time_avg_ms, "time_std_ms":time_std_ms}
PerformanceBenchmark.time_pipeline=time_pipeline

In [21]:
pb= PerformanceBenchmark(pipe,clinc["test"])
perf_metrics=pb.run_benchmark()

 Model size (MB) - 418.15
Average latency (ms) - 102.73 +\- 21.33
Accuracy on test set - 0.867
